In [127]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from datetime import time
import matplotlib.pyplot as pplot
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
import math
import random
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.cross_validation import ShuffleSplit, train_test_split

In [230]:
df_train = pd.read_csv('data_process_final/treino_final.csv')
df_teste = pd.read_csv('data_process_final/teste_final.csv')
df_teste_para_weekday = pd.read_csv('data_process_final/teste_final_para_weekday.csv')

In [231]:
df_remove = df_train.loc[(df_train['day'] >= 1) & (df_train['day'] <= 7) ]

df_train = df_train.drop(df_remove.index)

In [232]:
del df_train['volume_proximo']
del df_teste['volume_proximo']

In [233]:
del df_train['volume_proximo_2']
del df_teste['volume_proximo_2']

In [234]:
def adiciona_media_desvio_por_dia(df):
    df['media_volume_hora_dia'] = df.groupby(['direction', 'tollgate_id', 'day', 'hour'])["volume"].transform(np.mean)
    df['desvio_padrao_hora_dia'] = df.groupby(['direction', 'tollgate_id', 'day', 'hour'])["volume"].transform(np.std)
    df['desvio_padrao_hora_dia'].fillna(df.groupby(['direction', 'tollgate_id', 'day', 'hour'])["volume"].transform(np.mean), inplace=True)
    df['min_volume_hora_dia'] = df.groupby(['direction', 'tollgate_id', 'day', 'hour'])['volume'].transform(np.min)
    df['max_volume_hora_dia'] = df.groupby(['direction', 'tollgate_id', 'day', 'hour'])['volume'].transform(np.max)
    df['mediana_volume_hora_dia'] = df.groupby(['direction', 'tollgate_id', 'day', 'hour'])['volume'].transform(np.median)
    return df

In [235]:
df_train = adiciona_media_desvio_por_dia(df_train)
df_teste= adiciona_media_desvio_por_dia(df_teste)

In [236]:
def adiciona_media_desvio_por_janela_dia_semana_1(df):
    df['time'] = pd.to_datetime(df['time'], format = '%Y-%m-%d %H:%M:%S')
    df['date'] = df['time'].dt.date
    mask = (df['time'] > '2016-09-18 00:00:00') & (df['time'] < '2016-09-26 00:00:00')
    df = df.loc[mask]
    df['min_volume_weekday'] = df.groupby(['time_window', 'week', 'direction', 'tollgate_id'])["volume"].transform(np.min)
    df['max_volume_weekday'] = df.groupby(['time_window', 'week', 'direction', 'tollgate_id'])["volume"].transform(np.max)
    df['mediana_volume_weekday'] = df.groupby(['time_window', 'week', 'direction', 'tollgate_id'])["volume"].transform(np.median)
    df['media_volume_weekday'] = df.groupby(['time_window', 'week', 'direction', 'tollgate_id'])["volume"].transform(np.mean)
    df['desvio_padrao_weekday'] = df.groupby(['time_window', 'week', 'direction', 'tollgate_id'])["volume"].transform(np.std)
    df['desvio_padrao_weekday'].fillna(df.groupby(['time_window', 'week', 'direction', 'tollgate_id'])["volume"].transform(np.mean), inplace=True)
    return df

def adiciona_media_desvio_por_janela_dia_semana_2(df):
    
    df['time'] = pd.to_datetime(df['time'], format = '%Y-%m-%d %H:%M:%S')
    df['date'] = df['time'].dt.date
    mask2 = (df['time'] > '2016-09-18 00:00:00') & (df['time'] < '2016-10-10 00:00:00')
    df = df.loc[mask2]
    df['min_volume_weekday'] = df.groupby(['time_window', 'week', 'direction', 'tollgate_id'])["volume"].transform(np.min)
    df['max_volume_weekday'] = df.groupby(['time_window', 'week', 'direction', 'tollgate_id'])["volume"].transform(np.max)
    df['mediana_volume_weekday'] = df.groupby(['time_window', 'week', 'direction', 'tollgate_id'])["volume"].transform(np.median)
    df['media_volume_weekday'] = df.groupby(['time_window', 'week', 'direction', 'tollgate_id'])["volume"].transform(np.mean)
    df['desvio_padrao_weekday'] = df.groupby(['time_window', 'week', 'direction', 'tollgate_id'])["volume"].transform(np.std)
    df['desvio_padrao_weekday'].fillna(df.groupby(['time_window', 'week', 'direction', 'tollgate_id'])["volume"].transform(np.mean), inplace=True)
    
    return df

def adiciona_media_desvio_por_janela_dia_semana_3(df):
    
    df['time'] = pd.to_datetime(df['time'], format = '%Y-%m-%d %H:%M:%S')
    df['date'] = df['time'].dt.date
    mask3 = (df['time'] > '2016-09-18 00:00:00') & (df['time'] < '2016-10-17 00:00:00')
    df = df.loc[mask3]
    df['min_volume_weekday'] = df.groupby(['time_window', 'week', 'direction', 'tollgate_id'])["volume"].transform(np.min)
    df['max_volume_weekday'] = df.groupby(['time_window', 'week', 'direction', 'tollgate_id'])["volume"].transform(np.max)
    df['mediana_volume_weekday'] = df.groupby(['time_window', 'week', 'direction', 'tollgate_id'])["volume"].transform(np.median)
    df['media_volume_weekday'] = df.groupby(['time_window', 'week', 'direction', 'tollgate_id'])["volume"].transform(np.mean)
    df['desvio_padrao_weekday'] = df.groupby(['time_window', 'week', 'direction', 'tollgate_id'])["volume"].transform(np.std)
    df['desvio_padrao_weekday'].fillna(df.groupby(['time_window', 'week', 'direction', 'tollgate_id'])["volume"].transform(np.mean), inplace=True)
    return df

def adiciona_media_desvio_por_janela_dia_semana_4(df):   
    df['time'] = pd.to_datetime(df['time'], format = '%Y-%m-%d %H:%M:%S')
    df['date'] = df['time'].dt.date
    mask4 = (df['time'] > '2016-09-18 00:00:00') & (df['time'] < '2016-10-25 00:00:00')
    df = df.loc[mask4]
    df['min_volume_weekday'] = df.groupby(['time_window', 'week', 'direction', 'tollgate_id'])["volume"].transform(np.min)
    df['max_volume_weekday'] = df.groupby(['time_window', 'week', 'direction', 'tollgate_id'])["volume"].transform(np.max)
    df['mediana_volume_weekday'] = df.groupby(['time_window', 'week', 'direction', 'tollgate_id'])["volume"].transform(np.median)
    df['media_volume_weekday'] = df.groupby(['time_window', 'week', 'direction', 'tollgate_id'])["volume"].transform(np.mean)
    df['desvio_padrao_weekday'] = df.groupby(['time_window', 'week','direction', 'tollgate_id'])["volume"].transform(np.std)
    df['desvio_padrao_weekday'].fillna(df.groupby(['time_window', 'week', 'direction', 'tollgate_id'])["volume"].transform(np.mean), inplace=True)
    return df

def adiciona_media_desvio_por_janela_dia_semana_5_teste(df1, df2):
    df_list = [df1, df2]
    df = pd.concat(df_list, ignore_index=True)
    df['time'] = pd.to_datetime(df['time'], format = '%Y-%m-%d %H:%M:%S')
    df['date'] = df['time'].dt.date
    mask4 = (df['time'] >= '2016-10-18 00:00:00') & (df['time'] < '2016-11-01 00:00:00')
    df = df.loc[mask4]
    df['min_volume_weekday'] = df.groupby(['time_window', 'week', 'direction', 'tollgate_id'])["volume"].transform(np.min)
    df['max_volume_weekday'] = df.groupby(['time_window', 'week', 'direction', 'tollgate_id'])["volume"].transform(np.max)
    df['mediana_volume_weekday'] = df.groupby(['time_window', 'week', 'direction', 'tollgate_id'])["volume"].transform(np.median)
    df['media_volume_weekday'] = df.groupby(['time_window', 'week', 'direction', 'tollgate_id'])["volume"].transform(np.mean)
    df['desvio_padrao_weekday'] = df.groupby(['time_window', 'week','direction', 'tollgate_id'])["volume"].transform(np.std)
    df['desvio_padrao_weekday'].fillna(df.groupby(['time_window', 'week', 'direction', 'tollgate_id'])["volume"].transform(np.mean), inplace=True)
    
    
    return df

In [237]:
df_train_a_25 =adiciona_media_desvio_por_janela_dia_semana_1(df_train)
df_train_a_10 = adiciona_media_desvio_por_janela_dia_semana_2(df_train)
df_train_a_17 = adiciona_media_desvio_por_janela_dia_semana_3(df_train)
df_train_a_24 = adiciona_media_desvio_por_janela_dia_semana_4(df_train)
df_teste_a_24 = adiciona_media_desvio_por_janela_dia_semana_5_teste(df_teste_para_weekday, df_teste)

/home/matheuspds/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/matheuspds/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/home/matheuspds/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://panda

In [238]:
len_train1 = len(df_train_a_25)
len_train2 = len(df_train_a_10)
len_train3 = len(df_train_a_17)
len_train4 = len(df_train_a_24)

x1 = df_train_a_25.ix[:len_train1 - 1, 21:]
x2 = df_train_a_10.ix[:len_train2 - 1, 21:]
x3 = df_train_a_17.ix[:len_train3 - 1, 21:]
x4 = df_train_a_24.ix[:len_train4 - 1, 21:]

df_train_list = [x1, x2, x3, x4]

#df_train_list = [df_train_a_25[df_train_a_25[['min_volume_weekday']]], df_train_a_10[df_train_a_10['min_volume_weekday']]]
#df_train_para_agregar = pd.concat(df_train_list)


/home/matheuspds/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/home/matheuspds/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys
/home/matheuspds/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/home/matheuspds/miniconda3/lib/python3.7/site-packages/ipyk

In [239]:
df_train_para_agregar = pd.concat(df_train_list, ignore_index=True)

In [240]:
df_train['media_volume_weekday'] = df_train_para_agregar['media_volume_weekday']
df_train['min_volume_weekday'] = df_train_para_agregar['min_volume_weekday']
df_train['max_volume_weekday'] = df_train_para_agregar['max_volume_weekday']
df_train['desvio_padrao_weekday'] = df_train_para_agregar['desvio_padrao_weekday']
df_train['mediana_volume_weekday'] = df_train_para_agregar['mediana_volume_weekday']

In [229]:
df_train.to_csv('kkk.csv')

In [241]:
df_teste['min_volume_weekday'] = df_teste_a_24['min_volume_weekday']
df_teste['max_volume_weekday'] = df_teste_a_24['max_volume_weekday']
df_teste['mediana_volume_weekday'] = df_teste_a_24['mediana_volume_weekday']
df_teste['media_volume_weekday'] = df_teste_a_24['media_volume_weekday']
df_teste['desvio_padrao_weekday'] = df_teste_a_24['desvio_padrao_weekday']

In [ ]:
#media do volume do dia naquela tollgate naquela direcao

In [242]:
def medidas_volume_tollgate_direction_am_pm(df):
    df['min_volume_dia_am_pm'] = df.groupby(['day', 'direction', 'tollgate_id', 'am_pm'])["volume"].transform(np.min)
    df['max_volume_dia_am_pm'] = df.groupby(['day', 'direction', 'tollgate_id', 'am_pm'])["volume"].transform(np.max)
    df['mediana_volume_dia_am_pm'] = df.groupby(['day', 'direction', 'tollgate_id', 'am_pm'])["volume"].transform(np.median)
    df['media_volume_dia_am_pm'] = df.groupby(['day', 'direction', 'tollgate_id', 'am_pm'])["volume"].transform(np.mean)
    df['desvio_padrao_am_pm'] = df.groupby(['day','direction', 'tollgate_id', 'am_pm'])["volume"].transform(np.std)
    df['desvio_padrao_am_pm'].fillna(df.groupby(['day', 'direction', 'tollgate_id', 'am_pm'])["volume"].transform(np.mean), inplace=True)
    return df

In [243]:
df_train = medidas_volume_tollgate_direction_am_pm(df_train)
df_teste = medidas_volume_tollgate_direction_am_pm(df_teste)

In [170]:
#df_train_para_agregar.to_csv("huehue.csv", index=False)
df_teste.to_csv("huehue2.csv", index=False)

In [244]:
def feature_format():
    #pd_volume_train = pd_volume_train.set_index(['time'])
    #pd_volume_test = pd_volume_test.set_index(['time'])
    #volume_train = v_train.groupby(['time_window','tollgate_id','direction','date', 'hour']).size().reset_index().rename(columns = {0:'volume'})
    #volume_test = v_test.groupby(['time_window','tollgate_id','direction','date', 'hour']).size().reset_index().rename(columns = {0:'volume'})
    #print(volume_train)                
    x = pd.Series(df_train['time_window'].unique())
    s = pd.Series(range(len(x)),index = x.values)
    df_train['window_n'] = df_train['time_window'].map(s)
    df_teste['window_n'] = df_teste['time_window'].map(s)
#        print vol_test.tail()
    #volume_train['weekday'] = v_train['weekday']
    #volume_test['weekday'] = v_test['weekday']
    
    feature_train = df_train.drop('volume', axis = 1)
    feature_test = df_teste.drop('volume',axis = 1)
    values_train = df_train['volume'].values
    values_test = df_teste['volume'].values
    
    return feature_train, feature_test, values_train, values_test

In [245]:
feature_train, feature_test, values_train, values_test = feature_format()
feature_train = pd.concat([feature_train, pd.get_dummies(feature_train['tollgate_id'])], axis=1)
feature_test = pd.concat([feature_test, pd.get_dummies(feature_test['tollgate_id'])], axis=1)

In [246]:
regressor_cubic = RandomForestRegressor(n_estimators=500, max_depth=10, oob_score=True)

In [270]:
regressor_cubic.fit(feature_train[[1,2,3,'direction','week', 'weekend','volume_anterior', 'volume_anterior_2', 'am_pm', 'window_n', 'desvio_padrao_hora_dia', 'mediana_volume_hora_dia', 'max_volume_hora_dia', 'media_volume_hora_dia', 'min_volume_hora_dia', 'media_volume_dia_am_pm', 'max_volume_dia_am_pm', 'mediana_volume_dia_am_pm', 'min_volume_dia_am_pm', 'desvio_padrao_am_pm', 'media_volume_weekday', 'desvio_padrao_weekday', 'max_volume_weekday', 'min_volume_weekday', 'mediana_volume_weekday']], values_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
           oob_score=True, random_state=None, verbose=0, warm_start=False)

In [271]:
y_pred = regressor_cubic.predict(feature_test[[1,2,3,'direction','week', 'weekend','volume_anterior', 'volume_anterior_2', 'am_pm', 'window_n', 'desvio_padrao_hora_dia', 'mediana_volume_hora_dia', 'max_volume_hora_dia', 'media_volume_hora_dia', 'min_volume_hora_dia', 'media_volume_dia_am_pm', 'max_volume_dia_am_pm', 'mediana_volume_dia_am_pm', 'min_volume_dia_am_pm', 'desvio_padrao_am_pm', 'media_volume_weekday', 'desvio_padrao_weekday', 'max_volume_weekday', 'min_volume_weekday', 'mediana_volume_weekday']])

In [272]:
mean_absolute_percentage_error(values_test, y_pred)

0.13030441999133693

In [250]:
rmse = sqrt(mean_squared_error(y_pred, values_test))
rmse

8.717719916802679

In [273]:
# Create the parameter grid based on the results of random search 
param_grid = {
    'max_depth': [int(x) for x in np.linspace(10, 110, num = 11)],
    'max_features': [None],
    'n_estimators': [200, 300, 500, 800, 1200, 1500, 1800]
    #colocar um intervalo mais logico. Treino teste e validacao (tira uma parte do treino para validacao, e os
    # e eu nao posso fazer isso pra proxima janela de treino)
    #volume proximo nao pode ser utilizado
    # a media de dias da semana anterior
    # max altura, max numero de features, n_estimadores (100, 200, 300, 400)
    # max altura (80, 90,100, 110)
    # n_features deixa o numero de features fixo que eu tenho passa o auto
    # random_state tirar do decision tree
    #fazer isso
    #falar primeiro do decision tree antes mesmo do random forest
}
# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 4, n_jobs = -1, verbose = 2)

In [274]:
grid_search.fit(feature_train[[1,2,3,'direction','week', 'weekend','volume_anterior', 'volume_anterior_2', 'am_pm', 'window_n', 'desvio_padrao_hora_dia', 'mediana_volume_hora_dia', 'max_volume_hora_dia', 'media_volume_hora_dia', 'min_volume_hora_dia', 'media_volume_dia_am_pm', 'max_volume_dia_am_pm', 'mediana_volume_dia_am_pm', 'min_volume_dia_am_pm', 'desvio_padrao_am_pm', 'media_volume_weekday', 'desvio_padrao_weekday', 'max_volume_weekday', 'min_volume_weekday', 'mediana_volume_weekday']], values_train)

Fitting 4 folds for each of 77 candidates, totalling 308 fits
[CV] max_depth=10, max_features=None, n_estimators=200 ...............
[CV] max_depth=10, max_features=None, n_estimators=200 ...............
[CV] max_depth=10, max_features=None, n_estimators=200 ...............
[CV] max_depth=10, max_features=None, n_estimators=200 ...............
[CV]  max_depth=10, max_features=None, n_estimators=200, total=  15.7s
[CV] max_depth=10, max_features=None, n_estimators=300 ...............
[CV]  max_depth=10, max_features=None, n_estimators=200, total=  15.9s
[CV]  max_depth=10, max_features=None, n_estimators=200, total=  15.9s
[CV] max_depth=10, max_features=None, n_estimators=300 ...............
[CV] max_depth=10, max_features=None, n_estimators=300 ...............
[CV]  max_depth=10, max_features=None, n_estimators=200, total=  16.6s
[CV] max_depth=10, max_features=None, n_estimators=300 ...............
[CV]  max_depth=10, max_features=None, n_estimators=300, total=  23.8s
[CV] max_depth=

[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  9.7min


[CV]  max_depth=20, max_features=None, n_estimators=300, total=  37.8s
[CV] max_depth=20, max_features=None, n_estimators=500 ...............
[CV]  max_depth=20, max_features=None, n_estimators=300, total=  41.8s
[CV] max_depth=20, max_features=None, n_estimators=500 ...............
[CV]  max_depth=20, max_features=None, n_estimators=500, total= 1.0min
[CV] max_depth=20, max_features=None, n_estimators=800 ...............
[CV]  max_depth=20, max_features=None, n_estimators=500, total= 1.0min
[CV] max_depth=20, max_features=None, n_estimators=800 ...............
[CV]  max_depth=20, max_features=None, n_estimators=500, total= 1.1min
[CV] max_depth=20, max_features=None, n_estimators=800 ...............
[CV]  max_depth=20, max_features=None, n_estimators=500, total= 1.1min
[CV] max_depth=20, max_features=None, n_estimators=800 ...............
[CV]  max_depth=20, max_features=None, n_estimators=800, total= 1.7min
[CV] max_depth=20, max_features=None, n_estimators=1200 ..............
[CV]  

[CV]  max_depth=40, max_features=None, n_estimators=500, total= 1.1min
[CV] max_depth=40, max_features=None, n_estimators=800 ...............
[CV]  max_depth=40, max_features=None, n_estimators=500, total= 1.2min
[CV] max_depth=40, max_features=None, n_estimators=800 ...............
[CV]  max_depth=40, max_features=None, n_estimators=500, total= 1.1min
[CV] max_depth=40, max_features=None, n_estimators=800 ...............
[CV]  max_depth=40, max_features=None, n_estimators=500, total= 1.1min
[CV] max_depth=40, max_features=None, n_estimators=800 ...............
[CV]  max_depth=40, max_features=None, n_estimators=800, total= 1.7min
[CV] max_depth=40, max_features=None, n_estimators=1200 ..............
[CV]  max_depth=40, max_features=None, n_estimators=800, total= 1.7min
[CV] max_depth=40, max_features=None, n_estimators=1200 ..............
[CV]  max_depth=40, max_features=None, n_estimators=800, total= 1.9min
[CV] max_depth=40, max_features=None, n_estimators=1200 ..............
[CV]  

[CV]  max_depth=60, max_features=None, n_estimators=500, total= 1.1min
[CV] max_depth=60, max_features=None, n_estimators=800 ...............
[CV]  max_depth=60, max_features=None, n_estimators=500, total= 1.1min
[CV] max_depth=60, max_features=None, n_estimators=800 ...............
[CV]  max_depth=60, max_features=None, n_estimators=800, total= 1.7min
[CV] max_depth=60, max_features=None, n_estimators=1200 ..............
[CV]  max_depth=60, max_features=None, n_estimators=800, total= 1.7min
[CV] max_depth=60, max_features=None, n_estimators=1200 ..............


...

[CV]  max_depth=60, max_features=None, n_estimators=800, total= 1.9min
[CV] max_depth=60, max_features=None, n_estimators=1200 ..............
[CV]  max_depth=60, max_features=None, n_estimators=800, total= 1.8min
[CV] max_depth=60, max_features=None, n_estimators=1200 ..............
[CV]  max_depth=60, max_features=None, n_estimators=1200, total= 2.6min
[CV] max_depth=60, max_features=None, n_estimators=1500 ..............
[CV]  max_depth=60, max_features=None, n_estimators=1200, total= 2.6min
[CV] max_depth=60, max_features=None, n_estimators=1500 ..............
[CV]  max_depth=60, max_features=None, n_estimators=1200, total= 2.6min
[CV] max_depth=60, max_features=None, n_estimators=1500 ..............
[CV]  max_depth=60, max_features=None, n_estimators=1200, total= 2.9min
[CV] max_depth=60, max_features=None, n_estimators=1500 ..............
[CV]  max_depth=60, max_features=None, n_estimators=1500, total= 3.3min
[CV] max_depth=60, max_features=None, n_estimators=1800 ..............
[

[CV]  max_depth=80, max_features=None, n_estimators=1200, total= 2.6min
[CV] max_depth=80, max_features=None, n_estimators=1500 ..............
[CV]  max_depth=80, max_features=None, n_estimators=1200, total= 2.6min
[CV] max_depth=80, max_features=None, n_estimators=1500 ..............
[CV]  max_depth=80, max_features=None, n_estimators=1200, total= 2.8min
[CV] max_depth=80, max_features=None, n_estimators=1500 ..............
[CV]  max_depth=80, max_features=None, n_estimators=1200, total= 2.6min
[CV] max_depth=80, max_features=None, n_estimators=1500 ..............
[CV]  max_depth=80, max_features=None, n_estimators=1500, total= 3.2min
[CV] max_depth=80, max_features=None, n_estimators=1800 ..............
[CV]  max_depth=80, max_features=None, n_estimators=1500, total= 3.2min
[CV] max_depth=80, max_features=None, n_estimators=1800 ..............
[CV]  max_depth=80, max_features=None, n_estimators=1500, total= 3.2min
[CV] max_depth=80, max_features=None, n_estimators=1800 ..............

[CV] max_depth=100, max_features=None, n_estimators=1500 .............
[CV]  max_depth=100, max_features=None, n_estimators=1200, total= 2.7min
[CV] max_depth=100, max_features=None, n_estimators=1500 .............
[CV]  max_depth=100, max_features=None, n_estimators=1200, total= 3.0min
[CV] max_depth=100, max_features=None, n_estimators=1500 .............
[CV]  max_depth=100, max_features=None, n_estimators=1500, total= 3.4min
[CV] max_depth=100, max_features=None, n_estimators=1800 .............
[CV]  max_depth=100, max_features=None, n_estimators=1500, total= 3.4min
[CV] max_depth=100, max_features=None, n_estimators=1800 .............
[CV]  max_depth=100, max_features=None, n_estimators=1500, total= 3.4min
[CV] max_depth=100, max_features=None, n_estimators=1800 .............
[CV]  max_depth=100, max_features=None, n_estimators=1500, total= 3.8min
[CV] max_depth=100, max_features=None, n_estimators=1800 .............
[CV]  max_depth=100, max_features=None, n_estimators=1800, total=

[Parallel(n_jobs=-1)]: Done 308 out of 308 | elapsed: 154.5min finished


GridSearchCV(cv=4, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110], 'max_features': [None], 'n_estimators': [200, 300, 500, 800, 1200, 1500, 1800]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [275]:
grid_search.best_params_


{'max_depth': 10, 'max_features': None, 'n_estimators': 800}

In [36]:
print('Improvement of {:0.2f}%.'.format( 100 * (grid_accuracy - base_accuracy) / base_accuracy))

Improvement of 1.68%.


In [277]:
regressor_cubic = RandomForestRegressor(n_estimators=800, max_features=None, max_depth=10)

In [278]:
regressor_cubic.fit(feature_train[[1,2,3,'direction','week', 'weekend','volume_anterior', 'volume_anterior_2', 'am_pm', 'window_n', 'desvio_padrao_hora_dia', 'mediana_volume_hora_dia', 'max_volume_hora_dia', 'media_volume_hora_dia', 'min_volume_hora_dia', 'media_volume_dia_am_pm', 'max_volume_dia_am_pm', 'mediana_volume_dia_am_pm', 'min_volume_dia_am_pm', 'desvio_padrao_am_pm', 'media_volume_weekday', 'desvio_padrao_weekday', 'max_volume_weekday', 'min_volume_weekday', 'mediana_volume_weekday']], values_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features=None, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=800, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [279]:
y_pred = regressor_cubic.predict(feature_test[[1,2,3,'direction','week', 'weekend','volume_anterior', 'volume_anterior_2', 'am_pm', 'window_n', 'desvio_padrao_hora_dia', 'mediana_volume_hora_dia', 'max_volume_hora_dia', 'media_volume_hora_dia', 'min_volume_hora_dia', 'media_volume_dia_am_pm', 'max_volume_dia_am_pm', 'mediana_volume_dia_am_pm', 'min_volume_dia_am_pm', 'desvio_padrao_am_pm', 'media_volume_weekday', 'desvio_padrao_weekday', 'max_volume_weekday', 'min_volume_weekday', 'mediana_volume_weekday']])

In [280]:
mean_absolute_percentage_error(values_test, y_pred)

0.13019089509418424

In [281]:
rmse = sqrt(mean_squared_error(y_pred, values_test))
rmse

8.716885797477966

In [ ]:
##ADABOOSTING REGRESSOR

In [283]:
def ADABooster(param_grid, n_jobs): 
    estimator = AdaBoostRegressor() 
    cv = ShuffleSplit(feature_train[[1,2,3,'direction','week', 'weekend','volume_anterior', 'volume_anterior_2', 'am_pm', 'window_n', 'desvio_padrao_hora_dia', 'mediana_volume_hora_dia', 'max_volume_hora_dia', 'media_volume_hora_dia', 'min_volume_hora_dia', 'media_volume_dia_am_pm', 'max_volume_dia_am_pm', 'mediana_volume_dia_am_pm', 'min_volume_dia_am_pm', 'desvio_padrao_am_pm', 'media_volume_weekday', 'desvio_padrao_weekday', 'max_volume_weekday', 'min_volume_weekday', 'mediana_volume_weekday']].shape[0], test_size=0.2) 
    classifier = GridSearchCV(estimator=estimator, cv=4, param_grid=param_grid, n_jobs=n_jobs) 
    classifier.fit(feature_train[[1,2,3,'direction','week', 'weekend','volume_anterior', 'volume_anterior_2', 'am_pm', 'window_n', 'desvio_padrao_hora_dia', 'mediana_volume_hora_dia', 'max_volume_hora_dia', 'media_volume_hora_dia', 'min_volume_hora_dia', 'media_volume_dia_am_pm', 'max_volume_dia_am_pm', 'mediana_volume_dia_am_pm', 'min_volume_dia_am_pm', 'desvio_padrao_am_pm', 'media_volume_weekday', 'desvio_padrao_weekday', 'max_volume_weekday', 'min_volume_weekday', 'mediana_volume_weekday']], values_train) 
    print ("Best Estimator learned through GridSearch")
    print (classifier.best_estimator_)
    return cv, classifier.best_estimator_

In [284]:
param_grid={'n_estimators':[200, 300, 500, 800, 1200, 1500, 1800], 
            'learning_rate': [0.1, 0.05, 0.01, 0.005], 
            'loss':['linear', 'square', 'exponential']}
n_jobs=-1 

cv,best_est=ADABooster(param_grid, n_jobs)

Best Estimator learned through GridSearch
AdaBoostRegressor(base_estimator=None, learning_rate=0.01, loss='square',
         n_estimators=1200, random_state=None)


In [285]:
regr = AdaBoostRegressor(n_estimators=1200, learning_rate=0.01, loss='square')

In [286]:
regr.fit(feature_train[[1,2,3,'direction','week', 'weekend','volume_anterior', 'volume_anterior_2', 'am_pm', 'window_n', 'desvio_padrao_hora_dia', 'mediana_volume_hora_dia', 'max_volume_hora_dia', 'media_volume_hora_dia', 'min_volume_hora_dia', 'media_volume_dia_am_pm', 'max_volume_dia_am_pm', 'mediana_volume_dia_am_pm', 'min_volume_dia_am_pm', 'desvio_padrao_am_pm', 'media_volume_weekday', 'desvio_padrao_weekday', 'max_volume_weekday', 'min_volume_weekday', 'mediana_volume_weekday']], values_train) 

AdaBoostRegressor(base_estimator=None, learning_rate=0.01, loss='square',
         n_estimators=1200, random_state=None)

In [123]:
regr.feature_importances_

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 2.46036113e-05,
       0.00000000e+00, 4.90057580e-03, 3.37452797e-03, 0.00000000e+00,
       7.94052041e-03, 0.00000000e+00, 6.97806457e-03, 2.38786604e-02,
       9.50440266e-01, 2.46278120e-03])

In [287]:
y_pred_ada = regr.predict(feature_test[[1,2,3,'direction','week', 'weekend','volume_anterior', 'volume_anterior_2', 'am_pm', 'window_n', 'desvio_padrao_hora_dia', 'mediana_volume_hora_dia', 'max_volume_hora_dia', 'media_volume_hora_dia', 'min_volume_hora_dia', 'media_volume_dia_am_pm', 'max_volume_dia_am_pm', 'mediana_volume_dia_am_pm', 'min_volume_dia_am_pm', 'desvio_padrao_am_pm', 'media_volume_weekday', 'desvio_padrao_weekday', 'max_volume_weekday', 'min_volume_weekday', 'mediana_volume_weekday']])

In [132]:
regr.score(feature_train[['tollgate_id','direction','week', 'weekend','volume_anterior', 'volume_anterior_2', 'am_pm', 'window_n', 'desvio_padrao_hora_dia', 'mediana_volume_hora_dia', 'max_volume_hora_dia', 'media_volume_hora_dia', 'min_volume_hora_dia']], values_train)

0.9324050491518168

In [288]:
mean_absolute_percentage_error(values_test, y_pred_ada)

0.1732210121386972

In [289]:
rmse = sqrt(mean_squared_error(values_test, y_pred_ada))
rmse

10.576933295165214

In [50]:
rng = np.random.RandomState(1)
regr = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 50),
                         n_estimators=300, random_state = rng)

In [51]:
regr.fit(feature_train[['tollgate_id', 'direction','week', 'am_pm', 'volume_anterior', 'volume_anterior_2', 'volume_proximo', 'volume_proximo_2', 'avg_vol_dia_semana', 'desvio_padrao', 'window_n']], values_train)

y_pred = regr.predict(feature_test[['tollgate_id', 'direction','week', 'am_pm', 'volume_anterior', 'volume_anterior_2', 'volume_proximo', 'volume_proximo_2', 'avg_vol_dia_semana', 'desvio_padrao', 'window_n']])

In [52]:
rmse = sqrt(mean_squared_error(values_test, y_pred))
rmse

14.896148431565727

In [69]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true))

In [58]:
mean_absolute_percentage_error(values_test, y_pred)

0.20620277519594812

In [1]:
#GRADIENT BOOSTING

In [22]:
#Fazendo o GBRT COM PARAMETROS MAIS SIMPLES

In [24]:
gbrt=GradientBoostingRegressor(n_estimators=100) 
gbrt.fit(feature_train[['tollgate_id', 'direction','week', 'am_pm', 'volume_anterior', 'volume_anterior_2', 'volume_proximo', 'volume_proximo_2', 'avg_vol_dia_semana', 'desvio_padrao', 'window_n']], values_train)

y_pred_gbrt=gbrt.predict(feature_test[['tollgate_id', 'direction','week', 'am_pm', 'volume_anterior', 'volume_anterior_2', 'volume_proximo', 'volume_proximo_2', 'avg_vol_dia_semana', 'desvio_padrao', 'window_n']])

In [25]:
mean_absolute_percentage_error(values_test, y_pred_gbrt)

0.2620375772444925

In [290]:
def GradientBooster(param_grid, n_jobs): 
    estimator = GradientBoostingRegressor() 
    cv = ShuffleSplit(feature_train[[1,2,3,'direction','week', 'weekend','volume_anterior', 'volume_anterior_2', 'am_pm', 'window_n', 'desvio_padrao_hora_dia', 'mediana_volume_hora_dia', 'max_volume_hora_dia', 'media_volume_hora_dia', 'min_volume_hora_dia', 'media_volume_dia_am_pm', 'max_volume_dia_am_pm', 'mediana_volume_dia_am_pm', 'min_volume_dia_am_pm', 'desvio_padrao_am_pm', 'media_volume_weekday', 'desvio_padrao_weekday', 'max_volume_weekday', 'min_volume_weekday', 'mediana_volume_weekday']].shape[0], test_size=0.2) 
    classifier = GridSearchCV(estimator=estimator, cv=cv, param_grid=param_grid, n_jobs=n_jobs) 
    classifier.fit(feature_train[[1,2,3,'direction','week', 'weekend','volume_anterior', 'volume_anterior_2', 'am_pm', 'window_n', 'desvio_padrao_hora_dia', 'mediana_volume_hora_dia', 'max_volume_hora_dia', 'media_volume_hora_dia', 'min_volume_hora_dia', 'media_volume_dia_am_pm', 'max_volume_dia_am_pm', 'mediana_volume_dia_am_pm', 'min_volume_dia_am_pm', 'desvio_padrao_am_pm', 'media_volume_weekday', 'desvio_padrao_weekday', 'max_volume_weekday', 'min_volume_weekday', 'mediana_volume_weekday']], values_train) 
    print ("Best Estimator learned through GridSearch")
    print (classifier.best_estimator_)
    return cv, classifier.best_estimator_

In [ ]:
param_grid={'n_estimators':[200, 300, 500, 800, 1200, 1500, 1800], 
            'max_depth':[int(x) for x in np.linspace(10, 110, num = 11)],
            'max_features': [None]}
n_jobs=-1 

cv,best_est=GradientBooster(param_grid, n_jobs)

In [38]:
print ("Best Estimator Parameters") 
print("---------------------------" )
print ("n_estimators: %d" %best_est.n_estimators) 
print ("max_depth: %d" %best_est.max_depth) 
print ("Learning Rate: %.1f" %best_est.learning_rate) 
print ("min_samples_leaf: %d" %best_est.min_samples_leaf) 
print ("max_features: %.1f" %best_est.max_features) 

print ("Train R-squared: %.2f" %best_est.score(feature_train[['tollgate_id', 'direction','week', 'am_pm', 'volume_anterior', 'volume_anterior_2', 'volume_proximo', 'volume_proximo_2', 'avg_vol_dia_semana', 'desvio_padrao', 'window_n']], values_train) ) 

Best Estimator Parameters
---------------------------
n_estimators: 100
max_depth: 6
Learning Rate: 0.1
min_samples_leaf: 3
max_features: 1.0
Train R-squared: 0.97


In [40]:
estimator = best_est
estimator.fit(feature_train[['tollgate_id', 'direction','week', 'am_pm', 'volume_anterior', 'volume_anterior_2', 'volume_proximo', 'volume_proximo_2', 'avg_vol_dia_semana', 'desvio_padrao', 'window_n']], values_train) 

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=6, max_features=1.0,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=3,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [43]:
print ("Train R-squared: %.2f" %estimator.score(feature_train[['tollgate_id', 'direction','week', 'am_pm', 'volume_anterior', 'volume_anterior_2', 'volume_proximo', 'volume_proximo_2', 'avg_vol_dia_semana', 'desvio_padrao', 'window_n']], values_train)  )

print ("Test R-squared: %.2f" %estimator.score(feature_test[['tollgate_id', 'direction','week', 'am_pm', 'volume_anterior', 'volume_anterior_2', 'volume_proximo', 'volume_proximo_2', 'avg_vol_dia_semana', 'desvio_padrao', 'window_n']], values_test)) 

Train R-squared: 0.97
Test R-squared: 0.86


In [44]:
estimator = GradientBoostingRegressor(n_estimators=best_est.n_estimators, max_depth=best_est.max_depth, learning_rate=best_est.learning_rate, min_samples_leaf=best_est.min_samples_leaf, max_features=best_est.max_features)

In [45]:
estimator.fit(feature_train[['tollgate_id', 'direction','week', 'am_pm', 'volume_anterior', 'volume_anterior_2', 'volume_proximo', 'volume_proximo_2', 'avg_vol_dia_semana', 'desvio_padrao', 'window_n']], values_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=6, max_features=1.0,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=3,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [46]:
y_pred_gbrt_oficial = estimator.predict(feature_test[['tollgate_id', 'direction','week', 'am_pm', 'volume_anterior', 'volume_anterior_2', 'volume_proximo', 'volume_proximo_2', 'avg_vol_dia_semana', 'desvio_padrao', 'window_n']])

In [48]:
mean_absolute_percentage_error(values_test, y_pred_gbrt_oficial)

0.21357123313872148

In [49]:
rmse = sqrt(mean_squared_error(values_test, y_pred_gbrt_oficial))
rmse

14.16351207391079

In [251]:


regressor_cubic_g = GradientBoostingRegressor(n_estimators=250, learning_rate=0.05, min_samples_split=15,
                                                min_samples_leaf=2, max_depth=5, subsample=0.8,
                                                random_state=10, loss="lad")

In [260]:
regressor_cubic_g.fit(feature_train[[1,2,3,'direction','week', 'weekend','volume_anterior', 'volume_anterior_2', 'am_pm', 'window_n', 'desvio_padrao_hora_dia', 'mediana_volume_hora_dia', 'max_volume_hora_dia', 'media_volume_hora_dia', 'min_volume_hora_dia', 'media_volume_dia_am_pm', 'max_volume_dia_am_pm', 'mediana_volume_dia_am_pm', 'min_volume_dia_am_pm', 'desvio_padrao_am_pm']], values_train)
yhat = regressor_cubic_g.predict(feature_test[[1,2,3,'direction','week', 'weekend','volume_anterior', 'volume_anterior_2', 'am_pm', 'window_n', 'desvio_padrao_hora_dia', 'mediana_volume_hora_dia', 'max_volume_hora_dia', 'media_volume_hora_dia', 'min_volume_hora_dia', 'media_volume_dia_am_pm', 'max_volume_dia_am_pm', 'mediana_volume_dia_am_pm', 'min_volume_dia_am_pm', 'desvio_padrao_am_pm' ]])

In [261]:
mean_absolute_percentage_error(values_test, yhat)

0.13641016695648603

In [262]:
rmse = sqrt(mean_squared_error(values_test, yhat))
rmse

9.122407266598332